In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import codecs

import nltk
import string

nltk.download('punkt')
nltk.download('popular')


In [ ]:
def tokenize_text(raw_text: str):
  tokenized_str = nltk.word_tokenize(raw_text)
  tokens = [i.lower() for i in tokenized_str if (i not in string.punctuation)]
  stop_words = nltk.corpus.stopwords.words('english') 
  filtred_tokens = [i for i in tokens if(i not in stop_words)]
  return filtred_tokens 

# Загрузка данных

In [ ]:
# VOCAB_SIZE = 10000

# (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)

# print('Training entires: {}'.format(len(train_data)))
# print('======================================')
# print('Test entires: {}'.format(len(train_data)))
# print('======================================')
# print('Length examples: {}'.format([len(train_data[0]), len(train_data[1])]))
# print('======================================')
# print('Entry example: {}'.format(train_data[0]))

In [ ]:
!unzip '/content/archive' -d ''

Archive:  /content/archive.zip
  inflating: all_haiku.csv           


In [ ]:
df = pd.read_csv('/content/all_haiku.csv')
df.dropna()
df.columns = ['kek', 'first', 'second', 'third', 'source', 'hash']

In [ ]:
df['haiku'] = df['first'].map(str) + " " + df['second']
df['haiku'] = df['haiku'].map(str) + " " + df['third']

In [ ]:
df

,kek,first,second,third,source,hash,haiku
0,0,fishing boats,colors of,the rainbow,tempslibres,FISHINGBOATSCOLORSOFTHERAINBOW,fishing boats colors of the rainbow
1,1,ash wednesday--,trying to remember,my dream,tempslibres,ASHWEDNESDAYTRYINGTOREMEMBERMYDREAM,ash wednesday-- trying to remember my dream
2,2,snowy morn--,pouring another cup,of black coffee,tempslibres,SNOWYMORNPOURINGANOTHERCUPOFBLACKCOFFEE,snowy morn-- pouring another cup of black coffee
3,3,shortest day,flames dance,in the oven,tempslibres,SHORTESTDAYFLAMESDANCEINTHEOVEN,shortest day flames dance in the oven
4,4,haze,half the horse hidden,behind the house,tempslibres,HAZEHALFTHEHORSEHIDDENBEHINDTHEHOUSE,haze half the horse hidden behind the house
...,...,...,...,...,...,...,...
144118,118007,I'm not asking did,you say it nor clarify,what you said neither,twaiku,IMNOTASKINGDIDYOUSAYITNORCLARIFYWHATYOUSAIDNEI...,I'm not asking did you say it nor clarify wha...
144119,118008,You are truly a,moron or a liar I'm,inclined to think both,twaiku,YOUARETRULYAMORONORALIARIMINCLINEDTOTHINKBOTH,You are truly a moron or a liar I'm inclined ...
144120,118009,Ain't no selfie on,this earth that's gonna make me,like Theresa May,twaiku,AINTNOSELFIEONTHISEARTHTHATSGONNAMAKEMELIKETHE...,Ain't no selfie on this earth that's gonna ma...
144121,118010,is doing a great,job turning Independents,into Democrats,twaiku,ISDOINGAGREATJOBTURNINGINDEPENDENTSINTODEMOCRATS,is doing a great job turning Independents int...


In [ ]:
df.haiku.dropna(inplace=True)
df.drop(df.index[18799], axis=0, inplace=True)

In [ ]:
text = []
for num, i in enumerate(df.haiku.values):
  try:
    for j in tokenize_text(i):
        text.append(j)
  except Exception:
    print(num)
    break
text

['fishing',
 'boats',
 'colors',
 'rainbow',
 'ash',
 'wednesday',
 '--',
 'trying',
 'remember',
 'dream',
 'snowy',
 'morn',
 '--',
 'pouring',
 'another',
 'cup',
 'black',
 'coffee',
 'shortest',
 'day',
 'flames',
 'dance',
 'oven',
 'haze',
 'half',
 'horse',
 'hidden',
 'behind',
 'house',
 'low',
 'sun',
 'lady',
 'red',
 'high',
 'heels',
 'advent',
 'passing',
 'stranger',
 'farts',
 'tarn',
 'bubble',
 'ice',
 'snowflakes',
 'new',
 'asphalt',
 'holes',
 'crystal',
 'night',
 'gusts',
 'rain',
 'outside',
 'rain',
 'sound',
 'horse',
 'galloping',
 'leaves',
 'winter',
 'stars',
 'suddenly',
 'whiff',
 'perfume',
 'hungry',
 'half',
 'moon',
 'hidden',
 'rain',
 'another',
 'leaf',
 'sharia',
 'sound',
 'one',
 'hand',
 'clapping',
 'sound',
 'geese',
 'drowned',
 'sound',
 'train',
 'morning',
 'autumn',
 'sun',
 'shadow',
 'tombstones',
 'fly',
 'fishing',
 'sound',
 'wind',
 'reel',
 'december',
 'long',
 'shadow',
 'joins',
 'another',
 'end',
 'path',
 'snowflakes',
 'm

In [ ]:
len(text)

1070114

In [ ]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [ ]:
vocab = sorted(f7(text)) #извлечение символов unique
VOCAB_SIZE = len(vocab)

print('Vocab: {}'.format(vocab))
print('{} unique characters'.format(VOCAB_SIZE))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text]) #вместо символов - индексы
print('Example of the encoded text: {}'.format(text_as_int[:20]))

Vocab: ["''", "'06", "'_storm", "'accidental-ness", "'ai", "'all", "'an", "'and", "'annot", "'as", "'ath", "'ave", "'behold", "'bent", "'beshrew", "'blessings", "'bout", "'bove", "'buona", "'burned", "'but", "'buts", "'cars", "'cause", "'come", "'cos", "'crocus", "'crost", "'cry", "'cuckoo", "'cuss", "'cute", "'d", "'david", "'delia", "'du", "'eads", "'ee", "'em", "'er", "'ere", "'eukaleli", "'evangeline", "'father", "'food", "'for", "'forward", "'frederick", "'fuimus", "'gainst", "'gan", "'god", "'good", "'grace", "'gray", "'great", "'grew", "'had", "'hail", "'hear", "'hee-haw", "'her", "'hush", "'iccup", "'idden", "'if", "'ill", "'illimitable", "'im", "'in", "'int", "'into", "'is", "'january", "'kah", "'lady", "'let", "'ll", "'longside", "'look", "'lost", "'love", "'lover", "'m", "'maketh", "'mbianca", "'me", "'mid", "'midst", "'more", "'morrn", "'mother", "'mpacciata", "'mpaluda", "'mparadisa", "'mperador", "'mperio", "'mpromette", "'my", "'myself", "'n", "'native", "'nay", "'ncarca

In [ ]:
word_index = tf.keras.datasets.imdb.get_word_index()

word_index = {k: (v+3) for k, v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNKNOWN>'] = 2

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

#последовательность индексов в текст
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

#текст в последовательность индексов
def encode_review(text):
    words = text.lower().split()
    # words = ['<START>'] + words
    idxs = [word_index.get(word, word_index['<UNKNOWN>']) for word in words]
    return idxs

print('Example of decoded review: \n{}'.format(decode_review(train_data[0])))

Example of decoded review: 
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNKNOWN> is an amazing actor and now the same being director <UNKNOWN> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNKNOWN> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNKNOWN> to the two little boy's that played the <UNKNOWN> of norman and paul they were just brilliant children are often left out of the <UNKNOWN> list i think because the stars that play them all grown up are such a big profile for the whole film but thes

# Объеденим все отзывы в один текст (индексы)

In [ ]:
text_as_int = []
for num, item in enumerate(text):
  for word in text[num]:
    text_as_int.append(word)
text_as_int = np.array(text_as_int[:1000000])
len(text_as_int)

1000000

# Подготовка датасета
- Во время инфереса генератор будет работать следующим образо: сначала мы подаём входжной сигнал или входную последовательность (зерно), получаем новый выходной символ, а затем подаём его как входной символ и так далее(генерируем по однмоу символу за раз)

- А во время обучения мы будем обучать модель сразу целой последовательностью. Например, обученная модель должна по входу [f,i,r,s ...] выдавать [i,r,s,t, ...]
То есть ту же последовательность, но сдвинутую на 1 элемент. В данном случае это будет эквивалетно Maby-to-Many Sync(только во время обучения)

- Таким образом, зафиксируем длину рабочей цепочке SEQ_LEN (на которой будем обучаться), разрежем весь текст на цепочки длины SEQ_KEN+1 (остаток отбросим) и из каждой цепочки seq_len+1 получим пару цепочек seq_len, сдвинутых на 1 элемент. входная цепочка без последнего элемента и истинная(целевая) цепочка без первого элемента.

- Кроме того, зафиксируем размер батча для обучения И отбросим цепочки в конце, которые не могут наполнить полный батч(чтоб кол-во обучающих цепочек было кратно размеру батча)

In [ ]:
SEQ_LEN = 100
BATCH_SIZE = 64

input_seqs = []
target_seqs = []

num_seqs = len(text_as_int) // (SEQ_LEN+1)
for i in range(num_seqs):
    seq = text_as_int[i:i+SEQ_LEN+1]
    input_seqs.append(np.array(seq[:-1]))
    target_seqs.append(np.array(seq[1:]))

input_seqs = np.array(input_seqs)
target_seqs = np.array(target_seqs)

input_seqs = input_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]
target_seqs = target_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]

# print('Input: {} ...'.format([idx2char[i] for i in input_seqs[0][:15]]))
# print('Target: {} ...'.format([idx2char[i] for i in target_seqs[0][:15]]))

# Функция построения модели

In [ ]:
def build_model(batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, batch_input_shape=(batch_size, None)),
        tf.keras.layers.GRU(256, return_sequences=True, stateful=True), #STATEFUL - чтобы модель запоминаа предыдущи цепочки
        tf.keras.layers.Dropout(0.5),
        # tf.keras.layers.GRU(512, return_sequences=True, stateful=True),
        # tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(VOCAB_SIZE)
    ])
    model.build()
    return model

model = build_model(BATCH_SIZE)

In [15]:
EPOCHS = 100
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer='adam')

history = model.fit(
    input_seqs,
    target_seqs,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

Epoch 1/100
165/165 [==============================] - 48s 250ms/step - loss: 7.7399
Epoch 2/100
165/165 [==============================] - 42s 253ms/step - loss: 7.3788
Epoch 3/100
165/165 [==============================] - 41s 250ms/step - loss: 7.3813
Epoch 4/100
165/165 [==============================] - 42s 253ms/step - loss: 7.3838
Epoch 5/100
165/165 [==============================] - 41s 251ms/step - loss: 7.3848
Epoch 6/100
165/165 [==============================] - 41s 251ms/step - loss: 7.3853
Epoch 7/100
165/165 [==============================] - 42s 252ms/step - loss: 7.3265
Epoch 8/100
165/165 [==============================] - 42s 252ms/step - loss: 7.1621
Epoch 9/100
165/165 [==============================] - 42s 252ms/step - loss: 6.7887
Epoch 10/100
165/165 [==============================] - 42s 253ms/step - loss: 5.7123
Epoch 11/100
165/165 [==============================] - 42s 253ms/step - loss: 4.1452
Epoch 12/100
165/165 [==============================] - 42s 253

KeyboardInterrupt: ignored

# Создание модели дл инференса
- нужно создать такую же модель, но с размером батча=1 чтобы предсказывать текст по символу

In [17]:
model_inf = build_model(1)

for i in range(len(model_inf.layers)):
  for j in range(len(model_inf.layers[i].weights)):
    model_inf.layers[i].weights[j].assign(model.layers[i].weights[j])

# Функция генерации текста
- Создадим функцию, генерирующую текст по данной модели, входной цепочке и желаемому кол-ву слов
- Внутри функции сначала вызовем model.reset_states() для обнуления предыдущей истории состояния
- затем конвертируем входной текст seed и прогоним его через нашу модель. Нас интересует результат, соответствующих только последнему выходу(символу)
- как получить символ по предсказанию модели? можно использовать argmax, но в таком случае выход будет часто предсказуемым и иногда цепочка будет застрявать в цикличности
Поэтому чтобы сделать предсказание менее цикличным, воспользуемся распределнием и будем сэмплировать из него(то есть выбирать символ случайны образом в соответствии с его вероятнотью). Сделать это можно с помощью tf.random.categorical на вход categorical мы подаем всю матрицу pred(первое измерение - длина цепочки, второе - распределение по классам), а на выходе получаем список сэмлов, по одному дл каждого элемента последовательности.
а так как нас интересует лишь последний символ, необходимо выбрать только его с помощью индкса [-1] 
- Кроме того, можно ввести параметр температуры, с помощью которого можно управлять входным распредлением и таким образом влиять на непредсказуемость результата. Чем выше температура, тем непредсказуемей последний символ.
Например, если мы разделим логиты на ольшое число и мы бы применили софтмакс, то распределение стремилось бы к равномерному(у разных классов уравниваются шансы)

- это все одна итерация

In [22]:
def generate_text(model, seed, out_len):
    text_generated = []

    #обнуляем состояние модели
    model.reset_states()

    #конвернтируем цепочку 
    inp = np.array([char2idx[s] for s in seed])

    for i in range(out_len):

      #получаем предсказания для входной цепочки inp
      #pred - матрица размерности (длина цепочки, распределние по калссам)
      #на первой итерации цикла длина цепочки равна длине seed, а затем длина=1
      pred = model(inp[None, ...])[0]

      #для получчения символа сэмплируем из распределения
      #БОльшая температура соотв более случайному предсказанию
      temp = 1

      pred = pred / temp
      pred_c = tf.random.categorical(pred, num_samples=1)[-1][0].numpy()

      text_generated.append(idx2char[pred_c])

      #новый вход 
      inp = np.array([pred_c])
    return (seed + ' '.join(text_generated))


In [25]:
print(generate_text(model_inf, seed=u"hell", out_len=500))

hell... starting camp fire page fashion magazine new cellphone ... many people talk till battery dies city many stars unnamed late summer rain -- jump neon puddles breathe rosemary freight train -- night runs breath nighttime whoosh -- city train fades car park puddle -- walk sky around campfire wait milky way graffiti night -- stars sprayed everywhere city bound train -- book suitcase full stars autumn chill -- one little cricket sings moon cooler mornings -- honey slows spoon wild beaches winter time -- take home alone -- sound fork pasta bowl alongside train tracks -- secrets orchids moonlight stormy weather -- clarinet blues afternoon trains approaching -- someone platform reading moon sipping coffee slowly warms hands old memories train whistle distance another sleepless night smell rain young deer run across field white clouds glide safely mine fields winter solstice -- persian cat 's half- closed eyes need mountains hedgehog autumn deepens full moon another baby cries without ba

In [ ]:
history.history['loss']

[4.327425003051758,
 3.181915283203125,
 2.2687554359436035,
 1.673348307609558,
 1.2863234281539917,
 1.0277433395385742,
 0.847213089466095,
 0.7212099432945251,
 0.6265169978141785,
 0.5570146441459656,
 0.5045266151428223,
 0.46003058552742004,
 0.42710280418395996,
 0.39701417088508606,
 0.37491199374198914,
 0.35282325744628906,
 0.3367140293121338,
 0.32041165232658386,
 0.3078346848487854,
 0.2966328561306,
 0.28632858395576477,
 0.2756572961807251,
 0.2673700153827667,
 0.25963422656059265,
 0.25118792057037354,
 0.24626034498214722,
 0.23876874148845673,
 0.23290155827999115,
 0.22662022709846497,
 0.22257757186889648,
 0.21788020431995392,
 0.21339869499206543,
 0.20849786698818207,
 0.20628085732460022,
 0.20227397978305817,
 0.19738668203353882,
 0.1944240778684616,
 0.1901957094669342,
 0.18672969937324524,
 0.1852378100156784,
 0.18190637230873108,
 0.17880652844905853,
 0.1760978251695633,
 0.17411667108535767,
 0.17097292840480804,
 0.16823409497737885,
 0.166515588760